# ロバストMDPにおけるRectangularity

参考
* [Robust Markov Decision Process: Beyond Rectangularity](https://arxiv.org/abs/1811.00215)

表記
* 期待リターン：$R(\pi, \boldsymbol{P})=\mathbb{E}^{\pi, \boldsymbol{P}}\left[\sum_{t=0}^{\infty} \lambda^t r_{s_t a_t} \mid s_0 \sim \boldsymbol{p}_0\right]$


ロバストMDPは次のように遷移確率の不確実集合$\mathbb{P}$内での最悪ケースでの期待リターンを考えます．

$$
z^*=\max _{\pi \in \Pi^G} \min _{\boldsymbol{P} \in \mathbb{P}} R(\pi, \boldsymbol{P})
$$

不確実集合は何でも良いわけではありません．実際，任意の形式に対する最悪ケースを計算するのはNP困難です．（参考：[Robust Markov Decision Processes](https://optimization-online.org/wp-content/uploads/2010/05/2610.pdf)）

よく使われる不確実集合の構造として，$s$-rectangularと$sa$-rectangularがあります．

---

**$sa$-rectangular**

$$
\mathbb{P}=\times_{sa \in S A} \mathbb{P}_{sa}\quad \text{ where } \mathbb{P}_{sa} \in \Delta(S)
$$

---

---

**$s$-rectangular**

$$
\mathbb{P}=\times_{s \in S} \mathbb{P}_{s}\quad \text{ where } \mathbb{P}_{s} \in (\Delta(S))^{A}
$$

---

こうした$sa$や$s$-rectangularityは各$sa$や$s$について独立して最悪ケースの遷移確率を考えてしまうので，求めた解がかなり保守的になってしまう可能性があります．
例として，患者の健康状態を扱うMDPを考えてみましょう．$s$を患者の健康状態，$a$を処方箋とします．
患者の健康状態が異なっても，例えば遺伝や生理学的な特徴など，患者が持つ基礎因子が，異なる健康状態における健康状態の遷移に共通して影響を与える可能性があります．このような場合では，遷移確率$P_{sa}$でのゆらぎは他の状態での遷移のゆらぎに関係するべきです．

次のrectangularはこの問題を解決するために生まれました.
そのために，次のような不確実集合の一般系を導入します．

---

**Factor matrix uncertainty set**

$$
\mathbb{P}=\left\{\left(\sum_{i=1}^r u_{s a}^i w_{i, s^{\prime}}\right)_{s a s^{\prime}} \mid \boldsymbol{W}=\left(\boldsymbol{w}_1, \ldots, \boldsymbol{w}_r\right) \in \mathcal{W} \subseteq \mathbb{R}^{S \times r}\right\}
$$

ここで，$\boldsymbol{u}_1, \ldots, \boldsymbol{u}_S \in \mathbb{R}_{+}^{r \times A}$ は固定かつ既知であり，$\mathcal{W}$ は凸かつコンパクトな$\mathbb{R}_{+}^{S \times r}$の部分集合です．
$u$と$w$は次を満たすとします：

$$
\sum_{i=1}^r u_{s a}^i=1, \forall(s, a) \in \mathbb{S} \times \mathbb{A}, \sum_{s^{\prime}=1}^S w_{i, s^{\prime}}=1, \forall i \in[r]
$$

このFactor-matrix不確実集合は$s$と$sa$-rectangularityの一般系です．（論文のProposition 2.1）

---

これに対して，$r$-rectangularは次の形で導入されます．

---

**$r$-rectangularity**

$$
\mathcal{W}=\mathcal{W}^1 \times \ldots \times \mathcal{W}^r, \text { where } \mathcal{W}^1, \ldots, \mathcal{W}^r \subset \mathbb{R}_{+}^S
$$

---

試しにRectangularityの不確実集合について，挙動を確認してみます．

In [7]:
import numpy as np

S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ


# SA-rectangular

P = np.zeros((S, A, S))
def choose_worst_sa(s, a):  # 最悪ケースを選ぶやつの（仮）版です
    Psa = np.random.uniform(size=S)
    Psa = Psa / Psa.sum()
    return Psa

for s in range(S):
    for a in range(A):
        Psa = choose_worst_sa(s, a)  # 各saで独立した最悪ケースを選べます
        P[s, a] = Psa


# S-rectangular

def choose_worst_s(s):  # 最悪ケースを選ぶやつの（仮）版です
    Pas = np.random.uniform(size=(A, S))
    Pas = Pas / Pas.sum(axis=1, keepdims=True)
    return Pas

P = np.zeros((S, A, S))
for s in range(S):
    Pas = choose_worst_s(s)  # 各sで独立した最悪ケースを選べます
    P[s] = Pas


# R-rectangular

R = 4
U = np.random.uniform(size=(S, A, R))
U = U / U.sum(axis=-1, keepdims=True)
W = np.zeros((R, S))

def choose_worst_r(r):  # 最悪ケースを選ぶやつの（仮）版です
    Ps = np.random.uniform(size=S)
    Ps = Ps / Ps.sum(keepdims=True)
    return Ps

P = np.zeros((S, A, S))
for r in range(R):
    Ps = choose_worst_r(r)  # 各rで独立した最悪ケースを選べます
    Ur = U[:, :, r]  # S, A

    P_ = np.zeros((S, A, S))
    for s in range(S):
        for a in range(A):
            P_[s, a] = Ur[s, a] * Ps
    P += P_

$r$-rectangularityに対して，次の性質が成立します．

1. $(s, a)$-rectangularな不確実集合は$r$-rectangularでもあります．
2. $s$-rectangularではない$r$-rectangularな不確実集合が存在します
2. $sa$-rectangularではない$r$-rectangularな不確実集合が存在します
2. $r$-rectangularではない$s$-rectangularな不確実集合が存在します

よって，$r$-rectangularは$sa$-rectangularの一般系です．

## $r$-rectangularでの最適方策の導出

以下では$r$-rectangularにおいて最適方策を導出する話を見ていきます．
そのために，次の仮定を置きます：

---

**仮定**

$\mathcal{W}^1, \ldots, \mathcal{W}^r$はコンパクトな凸集合です．任意の$i \in [r]$とコストベクトル$c\in \mathbb{R}^S$について，
$$
\min _{\boldsymbol{w}_i \in \mathcal{W}^i} \boldsymbol{c}^{\top} \boldsymbol{w}_i
$$
の$\epsilon$最適解を$O\left(\operatorname{comp}\left(\mathcal{W}^i\right) \log \left(\epsilon^{-1}\right)\right)$の計算量で求められるとします．

---

この仮定を置くと，定常方策である最適方策が存在します．